In [12]:
from models import *
import pandas as pd
import os
import glob
import json

In [2]:
!rm database.db

rm: cannot remove ‘database.db’: No such file or directory


In [3]:
print db
db.create_all()

<SQLAlchemy engine='sqlite:////home/nabobalis/GitRepos/election-stats/database.db'>


Constituency : models.Constituency

Results : models.Results

Party : model.Party

Candidates : models.Candidates

In [4]:
db.session.rollback()

In [80]:
# JSON Files for Boundaries.
files = glob.glob('./election_json/boundaries/*.json')
files.sort()

In [8]:
!ls

app.py		  current_mp.xls  forms.py	  Procfile	    static
app.pyc		  database.db	  forms.pyc	  Procfile.dev	    templates
config.py	  election_json   models.py	  README.md	    _updated
config.pyc	  error.log	  models.pyc	  requirements.txt
constituency.xls  fabfile.py	  Populate.ipynb  screenshots


In [93]:
# XLS File with MP names.
data_1 = './current_mp.xls'
# Fetch names, sort and reindex.
names = pd.read_excel(data_1)
names = names.sort(columns=u'Constituency') # Sorted into constutuency order
names.index = range(0,len(names))

In [99]:
# XLS File with Constituency voting 2010.
data_2 = './constituency.xls'
cons = pd.read_excel(data_2)
cons

,General election results 2010,Turnout,Unnamed: 2,Party,Votes,Share,*lost deposit,change since 2005,Unnamed: 8
0,Aberavon [1],NaN,,,,,,,
1,50838,60.9,"Francis, H.*",Lab,16073,51.9,NaN,-,8.1
2,NaN,,"Davies, K.",LD,5034,16.3,NaN,+,2.5
3,NaN,,"Jones, C.Y. Ms.",Con,4411,14.2,NaN,+,4.1
4,NaN,,"Nicholls-Jones, P.",PC,2198,7.1,NaN,-,4.7
5,NaN,,"Edwards, K.A.",BNP,1276,4.1,*,NaN,
6,NaN,,"Tutton, A.",Ind,919,3,*,NaN,
7,NaN,,"Beany, C.",Bean,558,1.8,*,NaN,
8,NaN,,"Callan, J.",UKIP,489,1.6,*,NaN,
9,NaN,,,,-----,---,NaN,,


In [79]:
for afile in files:
    #Basic Name
    js = json.load(open(afile))
    name = str(js['properties']['PCON13NM']).lower()
    #geo = str(js['geometry'])
    con = Constituency(name=name)
    #con.boundary = geo
    #Results
    con.Results = Results
    con.Results.numvotes = 1
    con.Results.turnout = 1
    con.Results.year = 1
    # Candidates
    con.Candidates = Candidates
    con.Candidates.Party = Party
    con.Candidates.Party.name = ''
    db.session.add(con)

In [81]:
files

[]

In [14]:
db.session.commit()